In [1]:
#load pandas and numpy modules
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
 
#read raw data
df = pd.read_csv('Book1.csv')

if df.isnull().values.any():
    print ('missing values!')
    #raise ValueError
else:
    print('pass')

removed_mol = ["ASD007", "ASD012", "ASD019", "ASD021", "D372-0163"]
df = df[~df['MolID'].isin(removed_mol)]

df.head()

missing values!


,MolID,wt_A2_slope,pore_A2_slope,pd3_A2_slope,pd3pore_A2_slope,A2_pore/wt,A2_pd3/wt,A2_pd3-pore/wt,A2_10,A2_20,...,PMI3,RadiusOfGyration,SpherocityIndex,avg_asphericity,avg_acylindricity,avg_kappa2,avg_rg,avg_largest_principal_rg,avg_middle_principal_rg,avg_smallest_principal_rg
0,1500272,0.874,1.201,1.003,1.589,1.374142,1.147597,1.818078,13.25,29.46,...,8960.40,4.71,0.26,0.377154,0.212826,0.287922,0.526401,0.505270,0.457168,0.299362
1,1501007,1.224,2.024,2.130,2.240,1.653595,1.740196,1.830065,14.31,23.01,...,5322.33,4.43,0.13,0.310194,0.123611,0.349557,0.411174,0.391925,0.369452,0.216417
2,1501150,2.652,3.351,2.902,3.416,1.263575,1.094268,1.288084,40.09,79.97,...,4454.32,3.82,0.13,0.307322,0.156884,0.302154,0.421545,0.403439,0.370261,0.235260
3,1503100,10.150,10.846,8.314,9.200,1.068571,0.819113,0.906404,124.53,232.97,...,6975.32,4.50,0.21,0.402013,0.129149,0.435131,0.498724,0.478465,0.459285,0.236546
4,1503243,7.582,10.841,10.928,13.618,1.429834,1.441308,1.796096,106.38,200.59,...,6455.60,4.67,0.11,0.365638,0.124397,0.441792,0.454138,0.437384,0.416621,0.212002


In [2]:
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (recall_score,accuracy_score,confusion_matrix, f1_score, precision_score, 
                             auc,roc_auc_score,roc_curve, precision_recall_curve,classification_report)
from sklearn.model_selection import GridSearchCV,train_test_split

RF_param_grid = {"n_estimators" : [160, 320, 640, 1280],
                 "max_depth" : [None, 5, 10, 20, 40, 160, 320],
                 "min_samples_leaf" : [1, 2, 4, 8, 16, 32, 64],
                 "max_features":['auto', 5,10,20,30]
                }

RF = RandomForestClassifier(random_state=0,n_jobs=-1)

def data_split(X, Y, test_size, random_state=0):
    if not (type(X) is np.ndarray):
        X = X.to_numpy()
    if not (type(Y) is np.ndarray):
        Y = Y.to_numpy()
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = test_size, random_state=random_state)

    return X_train, X_test, Y_train, Y_test

In [4]:
descriptors = ['LabuteASA', 'NPR2', 'BalabanJ', 'MaxAbsEStateIndex', 'PMI3', 'MaxPartialCharge', 'MinEStateIndex', 'EState_VSA11', 'FractionCSP3', 'EState_VSA1', 'FpDensityMorgan2', 'avg_largest_principal_rg', 'avg_acylindricity', 'NPR1', 'NumAromaticHeterocycles', 'MinAbsEStateIndex', 'Kappa2', 'NumAliphaticHeterocycles', 'Chi3n', 'NumHAcceptors', 'EState_VSA3', 'NumHeteroatoms', 'NumHDonors', 'HallKierAlpha', 'NumAromaticCarbocycles', 'EState_VSA5', 'NumRotatableBonds', 'PMI1', 'EState_VSA7', 'EState_VSA2', 'EState_VSA9']

#Ycolumns = ["wt_A2_slope", "pore_A2_slope", "pd3_A2_slope", "pd3pore_A2_slope", "A2_10", "A2_20", "A2_40", "A2_80"]
Ycolumns = ["A2_pore/wt", "A2_pd3/wt", "A2_pd3-pore/wt"]

for column in Ycolumns:
    df_no_nan = df.dropna(subset=[column])
    Y = df_no_nan[column]
    #Y = df_no_nan.iloc[:,7]
    #Y = Y.dropna() # drop row if NaN in Y
    #b_class = {'Low': 0, 'High': 1}
    #Yb = Y.map(b_class)
    Yb = (Y > 1.2).astype(int) # threshold slope ratios
    #Yb = (Y > np.median(Y)).astype(int) # binary classification split by median
    #Yb = (Y > np.quantile(Y, 0.75)).astype(int)
    X_raw = df_no_nan.iloc[:,12:]
    #Yb.sum()
    scaler = RobustScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X_raw), columns=X_raw.columns)
    
    X_given = X_scaled[descriptors]

    ### Train-Test split ###
    X_train, X_test, Y_train, Y_test = data_split(X_given, Yb, 0.2)
    X_resampled, Y_resampled = SMOTE().fit_resample(X_train, Y_train) # resampling of train set

    gs = GridSearchCV(RF, param_grid = RF_param_grid, cv = 5, scoring='accuracy', n_jobs=-1)
    gs.fit(X_resampled, Y_resampled)
    print(f"#---{column}---")
    print("RF_tuned_params =", gs.best_params_)
    print("#Best score:", gs.best_score_)

#---A2_pore/wt---
RF_tuned_params = {'max_depth': 5, 'max_features': 30, 'min_samples_leaf': 4, 'n_estimators': 640}
#Best score: 0.74
#---A2_pd3/wt---
RF_tuned_params = {'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 1280}
#Best score: 0.8928571428571429
#---A2_pd3-pore/wt---
RF_tuned_params = {'max_depth': 5, 'max_features': 'auto', 'min_samples_leaf': 2, 'n_estimators': 160}
#Best score: 0.7382352941176471


In [19]:
#descriptors = ['FpDensityMorgan2', 'LabuteASA', 'MaxAbsEStateIndex', 'TPSA', 'EState_VSA7', 'Asphericity', 'Chi3v', 'MaxPartialCharge', 'PMI2', 'MaxAbsPartialCharge', 'NPR1', 'EState_VSA4', 'MinPartialCharge', 'Kappa3', 'MolLogP', 'NumHAcceptors', 'EState_VSA1', 'FractionCSP3', 'EState_VSA11', 'EState_VSA5', 'NumAliphaticCarbocycles', 'EState_VSA2', 'EState_VSA6', 'NumAromaticHeterocycles', 'EState_VSA9', 'BalabanJ', 'HallKierAlpha', 'EState_VSA3', 'MinAbsEStateIndex', 'EState_VSA8', 'InertialShapeFactor', 'NumAromaticCarbocycles', 'NumAliphaticHeterocycles'] # Valentin
#descriptors = ['NumAliphaticCarbocycles', 'Chi0n', 'NumHDonors', 'FpDensityMorgan2', 'MaxAbsEStateIndex', 'MaxPartialCharge', 'EState_VSA11', 'NPR1', 'EState_VSA2', 'PMI3', 'PMI1', 'EState_VSA8', 'MinEStateIndex', 'NPR2', 'NumHeteroatoms', 'EState_VSA1', 'Kappa3', 'EState_VSA3', 'MolLogP'] # 2D Rdkit
# descriptors = ['LabuteASA', 'NPR2', 'BalabanJ', 'MaxAbsEStateIndex', 'PMI3', 'MaxPartialCharge', 'MinEStateIndex', 'EState_VSA11', 'FractionCSP3', 'EState_VSA1', 'FpDensityMorgan2', 'avg_largest_principal_rg', 'avg_acylindricity', 'NPR1', 'NumAromaticHeterocycles', 'MinAbsEStateIndex', 'Kappa2', 'NumAliphaticHeterocycles', 'Chi3n', 'NumHAcceptors', 'EState_VSA3', 'NumHeteroatoms', 'NumHDonors', 'HallKierAlpha', 'NumAromaticCarbocycles', 'EState_VSA5', 'NumRotatableBonds', 'PMI1', 'EState_VSA7', 'EState_VSA2', 'EState_VSA9'] # 2D Rdkit + MD
# X_given = X_scaled[descriptors]

In [22]:
# ------ slopes, threshold = 3rd quatile ------
#---wt_A2_slope---
RF_tuned_params = {'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 640}
#Best score: 0.8552631578947368
#---pore_A2_slope---
RF_tuned_params = {'max_depth': 5, 'max_features': 10, 'min_samples_leaf': 2, 'n_estimators': 160}
#Best score: 0.8868421052631579
#---pd3_A2_slope---
RF_tuned_params = {'max_depth': 5, 'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 160}
#Best score: 0.8531578947368421
#---pd3pore_A2_slope---
RF_tuned_params = {'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 160}
#Best score: 0.8421052631578947

# ------- ratios, threshold = 1.2 -------
#---A2_pore/wt---
RF_tuned_params = {'max_depth': 5, 'max_features': 30, 'min_samples_leaf': 4, 'n_estimators': 640}
#Best score: 0.74
#---A2_pd3/wt---
RF_tuned_params = {'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 1280}
#Best score: 0.8928571428571429
#---A2_pd3-pore/wt---
RF_tuned_params = {'max_depth': 5, 'max_features': 'auto', 'min_samples_leaf': 2, 'n_estimators': 160}
#Best score: 0.7382352941176471

# ------ amplitudes, threshold = 3rd quatile ------
#---A2_10---
RF_tuned_params = {'max_depth': None, 'max_features': 10, 'min_samples_leaf': 1, 'n_estimators': 160}
#Best score: 0.8236842105263158
#---A2_20---
RF_tuned_params = {'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 640}
#Best score: 0.8347368421052632
#---A2_40---
RF_tuned_params = {'max_depth': None, 'max_features': 20, 'min_samples_leaf': 1, 'n_estimators': 320}
#Best score: 0.8031578947368422
#---A2_80---
RF_tuned_params = {'max_depth': None, 'max_features': 10, 'min_samples_leaf': 2, 'n_estimators': 320}
#Best score: 0.8136842105263158